<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Arthropod_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Arthropod Taxonomy Orders Object Detection Dataset](https://www.kaggle.com/datasets/mistag/arthropod-taxonomy-orders-object-detection-dataset/)



###### Lepidoptera = butterfies and moths
###### Hymenoptera = wasps, bees and ants
###### Hemiptera = true bugs (cicadas, aphids, shield bugs, ...)
###### Odonata = dragonflies
###### Diptera = fies
###### Araneae = spiders
###### Coleoptera = beetles

###### NOT IN DATASET
###### Orthoptera = grasshoppers

In [2]:
import os
import re
import sys
import json
import time
import numpy as np
import pprint as pp
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib as mpl
import concurrent.futures
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt


AUTO = tf.data.AUTOTUNE
pp = pp.PrettyPrinter()

!pip install --quiet tf-models-official==2.9.2
sys.path.append('PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER')# load all the metadata

import tensorflow_models as tfm
from tensorflow_models.vision import box_ops as box

     |████████████████████████████████| 2.1 MB 40.1 MB/s 
     |████████████████████████████████| 352 kB 70.2 MB/s 
     |████████████████████████████████| 238 kB 78.8 MB/s 
     |████████████████████████████████| 1.3 MB 70.8 MB/s 
     |████████████████████████████████| 118 kB 69.6 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 4.6 MB 56.3 MB/s 
     |████████████████████████████████| 662 kB 80.2 MB/s 
     |████████████████████████████████| 1.1 MB 75.8 MB/s 


In [ ]:
os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/'
!kaggle datasets download -d mistag/arthropod-taxonomy-orders-object-detection-dataset
!unzip *.zip && rm *.zip

# [Data Exploring](https://www.kaggle.com/code/mistag/starter-arthropod-taxonomy-orders-data-exploring)

The Arthropod Taxonomy Orders dataset is a collection of highres images annotated with labels from the taxanomy rank order. Annotations have been made with VoTT. VoTT stores all metadata in json files. In this kernel we will import all the metadata into DataFrames and export metadata to a variety of formats for object detection model training.
The dataset is distributed under CC BY-NC-SA 4.0

In [4]:
import glob

pfiles = glob.glob('/content/ArTaxOr/**/*.vott', recursive=True)

df = pd.DataFrame()
for p in pfiles:
  print(p)
  with open(p) as f:
    pdata = json.load(f)
    # print(pdata)
    df = df.append(pd.DataFrame(list(pdata['assets'].values())), ignore_index=True)

df['path'] = df['path'].str.replace('file:F:/', '')

/content/ArTaxOr/Araneae/annotations/Araneae.vott
/content/ArTaxOr/Odonata/annotations/Odonata.vott
/content/ArTaxOr/Diptera/annotations/Diptera.vott
/content/ArTaxOr/Lepidoptera/annotations/Lepidoptera.vott
/content/ArTaxOr/Hymenoptera/annotations/Hymenoptera.vott
/content/ArTaxOr/Hemiptera/annotations/Hemiptera.vott
/content/ArTaxOr/Coleoptera/annotations/Coleoptera.vott


In [5]:
df

,format,id,name,path,size,state,type
0,jpg,d0469bd539bde1b15b0acf2cd0d87f9d,0019ce6cda02.jpg,ArTaxOr/Araneae/0019ce6cda02.jpg,"{'width': 1100, 'height': 1588}",2,1
1,jpg,679107280d2f692152b436167c164018,003eb3db1665.jpg,ArTaxOr/Araneae/003eb3db1665.jpg,"{'width': 1280, 'height': 960}",2,1
2,jpg,f662e6835a6640dce287f3cf5f1373c6,00594c648f4d.jpg,ArTaxOr/Araneae/00594c648f4d.jpg,"{'width': 2048, 'height': 1370}",2,1
3,jpg,de01228a5521101e5fd9164a1a6cf8cc,007e1a3a7667.jpg,ArTaxOr/Araneae/007e1a3a7667.jpg,"{'width': 1128, 'height': 1349}",2,1
4,jpg,be2462acc54305b1a0a88c69cc17c273,00c50b891171.jpg,ArTaxOr/Araneae/00c50b891171.jpg,"{'width': 1344, 'height': 2048}",2,1
...,...,...,...,...,...,...,...
15371,jpg,a6b3e0784ae7e7b7217c039594bcca78,ff7ad41ae3f2.jpg,ArTaxOr/Coleoptera/ff7ad41ae3f2.jpg,"{'width': 907, 'height': 656}",2,1
15372,jpg,eb85b2a0363a14b8cd3e92c9c7c6ebf8,ff7f66cd4c82.jpg,ArTaxOr/Coleoptera/ff7f66cd4c82.jpg,"{'width': 1520, 'height': 1140}",2,1
15373,jpg,7a368cb2ae24af0a9cc9462477d6d363,ffa8620d4605.jpg,ArTaxOr/Coleoptera/ffa8620d4605.jpg,"{'width': 1536, 'height': 2048}",2,1
15374,jpg,3b7ecb51a97135974f263b751f3cc1cf,ffb21ba5c537.jpg,ArTaxOr/Coleoptera/ffb21ba5c537.jpg,"{'width': 800, 'height': 593}",2,1


In [6]:
labels = pd.DataFrame(list(pdata['tags']))[:-3]
labels

,name,color
0,Lepidoptera,#5db300
1,Coleoptera,#e81123
2,Hymenoptera,#6917aa
3,Diptera,#015cda
4,Araneae,#4894fe
5,Hemiptera,#257ffe
6,Odonata,#257ffe


In [7]:
p = 'ArTaxOr/Diptera/fff86b0ae807.jpg'
index = p[::-1].find('/')+1
p[:-index]

'ArTaxOr/Diptera'

In [8]:
anno=pd.DataFrame(columns=['label', 'label_idx', 'xres', 'yres', 'height', 'width', 'left', 'top', 
                           'right', 'bottom', 'area', 'xcenter', 'ycenter', 'blurred',
                           'occluded', 'truncated', 'file', 'id'])


#get all the image's annotation details (object data) from a json file and store it in a dataframe
for i in range(len(df)):  #15376 images(with details)
  p = df['path'][i]
  index = p[::-1].find('/')+1
  json_file=f'/content/{p[:-index]}/annotations/{df["id"][i]}-asset.json'

  if os.path.isfile(json_file):
    with open(json_file) as f:
      adata = json.load(f)
    xres=adata['asset']['size']['width']
    yres=adata['asset']['size']['height'] 
    for j in range(len(adata['regions'])):
            h=adata['regions'][j]['boundingBox']['height']/yres
            w=adata['regions'][j]['boundingBox']['width']/xres
            tags=adata['regions'][j]['tags']
            anno=anno.append({'label': tags[0],
                              'label_idx': labels[labels.name==tags[0]].index[0],
                              'xres': xres,
                              'yres': yres,
                              'height': h,
                              'width': w,                              
                              'left': adata['regions'][j]['boundingBox']['left']/xres,
                              'top': adata['regions'][j]['boundingBox']['top']/yres,
                              'right': adata['regions'][j]['boundingBox']['left']/xres+w,
                              'bottom': adata['regions'][j]['boundingBox']['top']/yres+h, 
                              'area': h*w,
                              'xcenter': adata['regions'][j]['boundingBox']['left']/xres+0.5*w,
                              'ycenter': adata['regions'][j]['boundingBox']['top']/yres+0.5*h,
                              'blurred': int(any(ele == '_blurred' for ele in tags)),
                              'occluded': int(any(ele == '_occluded' for ele in tags)),
                              'truncated': int(any(ele == '_truncated' for ele in tags)),
                              'file': adata['asset']['path'].replace('file:F:/',''),
                              'id': adata['asset']['id'],}, ignore_index=True)

In [9]:
anno.sample(5)

,label,label_idx,xres,yres,height,width,left,top,right,bottom,area,xcenter,ycenter,blurred,occluded,truncated,file,id
17451,Coleoptera,1,1862,1510,0.743295,0.830614,0.028118,0.194444,0.858732,0.937739,0.617391,0.443425,0.566092,0,0,0,ArTaxOr/Coleoptera/35740cd3ee05.jpg,5c4e0e30ceba0d331a4bd347c6a2b136
12451,Hymenoptera,2,1999,2048,0.498084,0.421001,0.137405,0.000000,0.558406,0.498084,0.209694,0.347905,0.249042,0,1,0,ArTaxOr/Hymenoptera/8aa762a85fc8.jpg,6607379d3e9c539148e6e3a0ab052c10
10402,Lepidoptera,0,5184,3456,0.476732,0.430737,0.250172,0.391594,0.680910,0.868327,0.205346,0.465541,0.629961,0,0,0,ArTaxOr/Lepidoptera/defe4353a751.jpg,f3e80ac65a66998ec7dca28af96f4169
12377,Hymenoptera,2,2048,1536,0.188697,0.130029,0.587284,0.277778,0.717313,0.466475,0.024536,0.652299,0.372126,0,0,0,ArTaxOr/Hymenoptera/85296e576e36.jpg,8c002355e76bb2c8fc07d8b20cc72c07
8328,Lepidoptera,0,4272,2848,0.385729,0.363887,0.356306,0.279909,0.720193,0.665638,0.140362,0.538249,0.472773,0,0,0,ArTaxOr/Lepidoptera/309899c278ac.jpg,71f4525fdbda1da4041657bcea0a00c1


In [10]:
# sns.relplot(x="width", y="height", hue="label", col="label", data=anno)

In [11]:
# sns.jointplot(x='width', y='height', data=anno.loc[anno['label'] == 'Diptera'])

In [12]:
sns.set(rc={'figure.figsize': (12,6)})
# sns.violinplot(x=anno['label'], y=anno['area'])

In [13]:
# graph=sns.countplot(data=anno, x='label')
# graph.set_xticklabels(graph.get_xticklabels(), rotation=90)

# for p in graph.patches:
#   height = p.get_height()
#   graph.text(p.get_x() + p.get_width()/2, height + 0.1, height, ha='center')

In [14]:
temp = 'truncated'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
# sns.countplot(x=temp, hue='label', data=df2)

In [15]:
temp = 'blurred'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
# sns.countplot(x=temp, hue='label', data=df2)

In [16]:
temp = 'occluded'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
# sns.countplot(x=temp, hue='label', data=df2)

## Image exploration


In [17]:
def attribution(fname):
    img = Image.open(fname)
    exif_data = img._getexif()
    img.close()
    if len(exif_data[315]) > 0:
        s='Photo: '+exif_data[315]
    else:
        s=exif_data[37510][8:].decode('ascii')
    return s

def plot_img(axes, image_df, highlight=True):
    image_file_name='/content/'+image_df.iloc[0].file
    im = Image.open(image_file_name)
    im.thumbnail((300,300),Image.ANTIALIAS)  #A small image representation of a larger image
    draw = ImageDraw.Draw(im)
    xres, yres = im.size[0], im.size[1] # X, Y Resolution
    for i in range(len(image_df)):
        selected_img = image_df.iloc[i]
        if highlight==True:
            color=(255, 0, 0) if i == 0 else (128, 128, 128)          
        else:
            color=labels[labels.name == selected_img.label].color.iloc[0]
        
        #draw a boundingbox
        draw.rectangle([int(selected_img['left']*xres),
                        int(selected_img['top']*yres),
                        int(selected_img['right']*xres),
                        int(selected_img['bottom']*yres)], outline=color, width=2)
        
    plt.setp(axes, xticks=[], yticks=[])  #remove the grids
    axes.set_title(image_df.iloc[0].label+'\n'+attribution(image_file_name))
    plt.imshow(im)

In [18]:
fig = plt.figure(figsize=(16,26))
for i in range(len(labels)):
  for j in range(3):
    #Select 3 images with the largest area
    # ldf=anno[anno.label == labels.name[i]].nlargest(3, 'area') 

    #Select images with the most objects
    a=anno[anno.label == labels.name[i]]['id'].value_counts()
    ldf=anno[anno.id == a.index[j]]
    # axes = fig.add_subplot(len(labels), 3, 1+i*3+j)
    # plot_img(axes, anno[anno.id == ldf.iloc[j].id].sort_values(by=['area'], ascending=False), highlight=False)

<Figure size 1152x1872 with 0 Axes>

# Metadata export

export metadata to various formats for object detection model training


## CSV

In [ ]:
header = ['file', 'label', 'height', 'width', 'left', 'top', 'right', 'bottom'] # change as required
anno.to_csv('./ArTaxOr.csv', index=False, columns = header) 

## Pascal VOC

Pascal VOC files are xml format, and there is one xml file per image file, with same name.

In [ ]:
!{sys.executable} -m pip install pascal_voc_writer
from pascal_voc_writer import Writer

if not os.path.exists('voc'):
    os.mkdir('voc')

for i in range(len(df)):
    ldf=anno[anno.id == df.id[i]].reset_index()
    p=df.path[i].split('/')
    image_xml = p[2].replace('.jpg','.xml') 
    width, height = ldf.xres[0], ldf.yres[0]
    
    writer = Writer(df.path[i], width, height)
    for j in range(len(ldf)):
        writer.addObject(ldf.label[j], 
                         int(ldf.left[j]*width), 
                         int(ldf.top[j]*height), 
                         int(ldf.right[j]*width),
                         int(ldf.bottom[j]*height))
    writer.save(f'./voc/{image_xml}')
# print(os.listdir("./voc"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Darknet YOLOv3
Darknet expects one annotation file per image file. Each object is described by:

`class x_center y_center width height`

In [ ]:
if not os.path.exists('labels'):
    os.mkdir('labels')

for i in range(len(df)):
    ldf=anno[anno.id == df.id[i]].reset_index()
    p=df.path[i].split('/') 
    image_txt = p[2].replace('.jpg','.txt') 
    
    file=open(f'./labels/{image_txt}','w')
    for j in range(len(ldf)):
        label_name=labels[labels.name == ldf.label[j]].index.to_list()
        file.write(f'{label_name[0]} {ldf.xcenter[j]} {ldf.ycenter[j]} {ldf.width[j]} {ldf.height[j]}\n')
    file.close()
# print(os.listdir("./labels"))

## Pickle

In [19]:
labels.to_pickle('./ArTaxOr_labels.pkl')
df.to_pickle('./ArTaxOr_filelist.pkl')
anno.to_pickle('./ArTaxOr_objects.pkl')

## [TensorFlow TFRecords](https://www.kaggle.com/code/mistag/tensorflow-tfrecords-demystified/notebook)

#### Protocol buffers
TFRecords are Protocol buffers. Protocol buffers are a way of serializing structured data in a compact and efficient way. A Protocol buffer message is defined by a .proto file. Example:

```
syntax = "proto3"; // define protocol version

// Define "Employee" message with 3 fields
message Employee {
 string name = 1; // a unique number is assigned to each field
 int32 company_id = 2;
 string address = 3;
}

```


A single .proto-file can define multiple messages and messages can be nested. The beauty of protocol buffers is that the .proto-files can be compiled into a library (language of choice) with access functions using the protoc compiler. But for TFRecords the access functions we need are already part of TensorFlow, so no need to worry about compiling .proto files.

### TFRecord format with simple data
TFRecords are made to support just about any type of data, and that means nesting basic features into a hierarchy of features.TFRecords supports only three different data types:

bytes
float
int64

So any data must be converted into one of these three types.

In [20]:
# tf.train.Feature can be used to convert data into lists of these types.
int_list = tf.train.Feature(int64_list=tf.train.Int64List(value=np.arange(8)))
int_list

int64_list {
  value: 0
  value: 1
  value: 2
  value: 3
  value: 4
  value: 5
  value: 6
  value: 7
}

In [21]:
float_list = tf.train.Feature(float_list=tf.train.FloatList(value=np.random.rand(10)))
float_list

float_list {
  value: 0.6677202582359314
  value: 0.527971625328064
  value: 0.04802408441901207
  value: 0.578150749206543
  value: 0.7106977105140686
  value: 0.19086655974388123
  value: 0.11398348212242126
  value: 0.5690155625343323
  value: 0.9284249544143677
  value: 0.8978065252304077
}

In [22]:
bytes_list = tf.train.Feature(bytes_list=tf.train.BytesList(value=['hello, my name is Esmail'.encode()]))
bytes_list

bytes_list {
  value: "hello, my name is Esmail"
}

In [23]:
features = tf.train.Features(feature={
    'integers': int_list,
    'float': float_list,
    'description': bytes_list
})
features

feature {
  key: "description"
  value {
    bytes_list {
      value: "hello, my name is Esmail"
    }
  }
}
feature {
  key: "float"
  value {
    float_list {
      value: 0.6677202582359314
      value: 0.527971625328064
      value: 0.04802408441901207
      value: 0.578150749206543
      value: 0.7106977105140686
      value: 0.19086655974388123
      value: 0.11398348212242126
      value: 0.5690155625343323
      value: 0.9284249544143677
      value: 0.8978065252304077
    }
  }
}
feature {
  key: "integers"
  value {
    int64_list {
      value: 0
      value: 1
      value: 2
      value: 3
      value: 4
      value: 5
      value: 6
      value: 7
    }
  }
}

The last step is to create an example TFRecord. So, example is not a very good name, but that is what the TFRecord is called. We simply pass the list of features to `tf.train.Example.`

In [24]:
tf_record = tf.train.Example(features=features)
tf_record

features {
  feature {
    key: "description"
    value {
      bytes_list {
        value: "hello, my name is Esmail"
      }
    }
  }
  feature {
    key: "float"
    value {
      float_list {
        value: 0.6677202582359314
        value: 0.527971625328064
        value: 0.04802408441901207
        value: 0.578150749206543
        value: 0.7106977105140686
        value: 0.19086655974388123
        value: 0.11398348212242126
        value: 0.5690155625343323
        value: 0.9284249544143677
        value: 0.8978065252304077
      }
    }
  }
  feature {
    key: "integers"
    value {
      int64_list {
        value: 0
        value: 1
        value: 2
        value: 3
        value: 4
        value: 5
        value: 6
        value: 7
      }
    }
  }
}

 write this record to a file using tf.io.TFRecordWriter

In [25]:
fname='example1.tfrecord'
with tf.io.TFRecordWriter(fname) as writer:
    writer.write(tf_record.SerializeToString())
print(f"Size of {fname} is {os.path.getsize(fname)} bytes")

Size of example1.tfrecord is 144 bytes


Next, read the file back in and access the data using tf.data.TFRecordDataset and tf.train.Example.FromString:

In [26]:
dataset = tf.data.TFRecordDataset(fname)
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())
parsed

features {
  feature {
    key: "description"
    value {
      bytes_list {
        value: "hello, my name is Esmail"
      }
    }
  }
  feature {
    key: "float"
    value {
      float_list {
        value: 0.6677202582359314
        value: 0.527971625328064
        value: 0.04802408441901207
        value: 0.578150749206543
        value: 0.7106977105140686
        value: 0.19086655974388123
        value: 0.11398348212242126
        value: 0.5690155625343323
        value: 0.9284249544143677
        value: 0.8978065252304077
      }
    }
  }
  feature {
    key: "integers"
    value {
      int64_list {
        value: 0
        value: 1
        value: 2
        value: 3
        value: 4
        value: 5
        value: 6
        value: 7
      }
    }
  }
}

Access each feature using `parsed.features.feature['<feature name>']`



In [27]:
parsed.features.feature['description'].bytes_list.value[0].decode()

'hello, my name is Esmail'

In [28]:
parsed.features.feature['integers'].int64_list.value[:]

[0, 1, 2, 3, 4, 5, 6, 7]

In [29]:
parsed.features.feature['float'].float_list.value[:]

[0.6677202582359314,
 0.527971625328064,
 0.04802408441901207,
 0.578150749206543,
 0.7106977105140686,
 0.19086655974388123,
 0.11398348212242126,
 0.5690155625343323,
 0.9284249544143677,
 0.8978065252304077]

### TFRecords with image data

To make input function efficient, all preprocessing steps should be performed when creating TFRecords, otherwise the input function must repeat preprocessing each time the training passed over the TFRecord. Typical preprocessing steps are:

#### Decode image from jpg etc.
#### Resize image
#### Convert to float
#### Normalize to [0,1] range

Augumentation is not something to do in a TFRecord, since agumentation could be different each time training passes over the data

In [30]:
import hashlib
from io import BytesIO
from PIL import Image, ImageFont, ImageDraw
ARTAXOR_PATH = '/content/'

pickles='/content/'
objectdf=pd.read_pickle(pickles+'ArTaxOr_objects.pkl')
labels=pd.read_pickle(pickles+'ArTaxOr_labels.pkl')
objectdf.sample(5)

,label,label_idx,xres,yres,height,width,left,top,right,bottom,area,xcenter,ycenter,blurred,occluded,truncated,file,id
2768,Odonata,6,2047,1399,0.441978,0.360690,0.442759,0.235621,0.803448,0.677598,0.159417,0.623103,0.456609,0,0,0,ArTaxOr/Odonata/3dfb0b15d471.jpg,5c9df99a1658f495cec1ae11c1dd49c0
17886,Coleoptera,1,1451,2048,0.565134,0.705405,0.212627,0.219349,0.918032,0.784483,0.398649,0.565329,0.501916,0,0,0,ArTaxOr/Coleoptera/59d7c2c9c772.jpg,2d22cb18d6ccd58db98a406c45243d35
3813,Odonata,6,2048,1365,0.929907,0.857217,0.024013,0.070093,0.881231,1.000000,0.797132,0.452622,0.535047,0,1,1,ArTaxOr/Odonata/8b9055492e62.jpg,44fd20c34f64c4b6e5b27c3c9cd3baa5
11322,Hymenoptera,2,1312,1056,0.272989,0.348497,0.316921,0.294061,0.665418,0.567050,0.095136,0.491170,0.430556,0,0,0,ArTaxOr/Hymenoptera/2e8b57f68c5e.jpg,d4f20504e7b2fb0351896ad91cdf4ae8
9048,Lepidoptera,0,4272,2848,0.455016,0.299104,0.090283,0.338854,0.389387,0.793870,0.136097,0.239835,0.566362,0,0,0,ArTaxOr/Lepidoptera/7899b796e05b.jpg,d7ef598f897f3dc6bf86d2341ef27e58


Define a function that creates a tf.train.Example from an image and the objects contained within.

    Note that TensorFlow Object Detection API expects jpeg-encoded image data

, so the only preprocessing to be done is resize (optional) before the image is re-encoded (via a memory buffer using BytesIO).


The input to this function is a DataFrame that contains one row per object and the columns shown above. This record is generous with features, but that is to make sure that the TF Object Detection API will successfully run the evaluation phase during training.

In [31]:
# Fetch attribution string from image EXIF data
def get_attribution(file):
    with Image.open(file) as img:
        exif_data = img._getexif()
    
    source ='Photo: unknown'
    if exif_data is not None:
        if 37510 in exif_data and len(exif_data[37510]) > 0:
            source = exif_data[37510][8:].decode('ascii')
        if 315 in exif_data and len(exif_data[315]) > 0:
            source = 'Photo: ' + exif_data[315]
    return source

# Create example for TensorFlow Object Detection API
def create_tf_example(imagedf, longest_edge=1024):  
    fname = ARTAXOR_PATH+imagedf.file.iloc[0]  #exclude image path 
    filename=fname.split('/')[-1]              #exclude image name
    by = get_attribution(fname)                #exclude EXIF data
    img = Image.open(fname, "r")

    # resize image if larger that longest edge while keeping aspect ratio
    if max(img.size) > longest_edge:
        img.thumbnail((longest_edge, longest_edge), Image.ANTIALIAS)
    
    width, height = img.size #(w, h)

    buf= BytesIO()
    img.save(buf, format= 'JPEG') # encode to jpeg in memory
    encoded_image_data= buf.getvalue()
    image_format = b'jpeg'
    
    source_id = filename.split('.')[0]
    license = 'CC BY-NC-SA 4.0'
    # A hash of the image is used in some frameworks
    key = hashlib.sha256(encoded_image_data).hexdigest() 

    # object bounding boxes 
    # List of normalized coordinates in bounding box (1 per box)
    xmins, xmaxs, ymins, ymaxs = imagedf[['left', 'right', 'top', 'bottom']].values.T

    # List of string class name & id of bounding box (1 per box)
    object_count = len(imagedf)
    classes_text = []
    classes = []
    for i in range(object_count):
        classes_text.append(imagedf.label.iloc[i].encode())
        classes.append(1 + imagedf.label_idx.iloc[i])

    #??????????????????????????????????????????????????????????????????
    
    # unused features from Open Image 
    depiction = np.zeros(object_count, dtype=int)
    group_of = np.zeros(object_count, dtype=int)
    occluded = imagedf.occluded.values #also Pascal VOC
    truncated = imagedf.truncated.values # also Pascal VOC


    # Pascal VOC
    view_text = []
    for i in range(object_count):
        view_text.append('frontal'.encode())
    difficult = np.zeros(object_count, dtype=int)

    tf_record = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[source_id.encode()])),
        'image/license': tf.train.Feature(bytes_list=tf.train.BytesList(value=[license.encode()])),
        'image/by': tf.train.Feature(bytes_list=tf.train.BytesList(value=[by.encode()])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image_data])),
        'image/key/sha256': tf.train.Feature(bytes_list=tf.train.BytesList(value=[key.encode()])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_format])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
        'image/object/depiction': tf.train.Feature(int64_list=tf.train.Int64List(value=depiction)),
        'image/object/group_of': tf.train.Feature(int64_list=tf.train.Int64List(value=group_of)),
        'image/object/occluded': tf.train.Feature(int64_list=tf.train.Int64List(value=occluded)),
        'image/object/truncated': tf.train.Feature(int64_list=tf.train.Int64List(value=truncated)),
        'image/object/difficult': tf.train.Feature(int64_list=tf.train.Int64List(value=difficult)),
        'image/object/view': tf.train.Feature(bytes_list=tf.train.BytesList(value=view_text))
    }))
    return tf_record

In [32]:
#An Example of creating tf record
sample_file='ArTaxOr/Hemiptera/00600a41bcaf.jpg'
imagedf=objectdf[objectdf.file == sample_file]
tfr=create_tf_example(imagedf)
# print(tfr)
fname='./image_ex1.tfrecord'
with tf.io.TFRecordWriter(fname) as writer:
    writer.write(tfr.SerializeToString())
print(f"Size of {fname} is {fname, os.path.getsize(fname)//1024} kbytes")

Size of ./image_ex1.tfrecord is ('./image_ex1.tfrecord', 95) kbytes


load back in and visualize the Tensordlow records contents.

In [33]:
font = ImageFont.load_default()
def bbox(img, xmin, ymin, xmax, ymax, color, width, label, score):
    draw = ImageDraw.Draw(img)

    xres, yres = img.size  #(w, h)

    #box[0] = xmin*xres
    box = np.multiply([xmin, ymin, xmax, ymax], [xres, yres, xres, yres]).astype(int).tolist()

    txt = f" {label}: {round(score, 1)}" if score >= 0. else f" {label}"
    ts = draw.textsize(txt, font=ImageFont.load_default())
    draw.rectangle(box, outline=color, width=width)

    #######????????    
    if len(label) > 0:
        if box[1] >= ts[1]+3:
            xsmin, ysmin = box[0],                box[1]-ts[1]-3
            xsmax, ysmax = box[0] + ts[0] + 2,    box[1]
        else:
            xsmin, ysmin = box[0],               box[3]
            xsmax, ysmax = box[0] + ts[0] + 2,   box[3] + ts[1] + 1
            
        draw.rectangle([xsmin, ysmin, xsmax, ysmax], fill=color)
        draw.text((xsmin, ysmin), txt, font=font, fill='white')

In [34]:
def plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label, by):
    for i in range(len(xmin)):
        color=labels.color[class_label[i]-1]
        bbox(img, xmin[i], ymin[i], xmax[i], ymax[i], color, 5, classes[i].decode(), -1)
    plt.setp(axes, xticks=[], yticks=[])
    axes.set_title(by)
    plt.imshow(img)

In [35]:
# load tfrecord
fname='image_ex1.tfrecord'
dataset = tf.data.TFRecordDataset(fname)
img_example = next(iter(dataset))
img_parsed = tf.train.Example.FromString(img_example.numpy())
# only extract features we will actually use
xmin=img_parsed.features.feature['image/object/bbox/xmin'].float_list.value[:]
xmax=img_parsed.features.feature['image/object/bbox/xmax'].float_list.value[:]
ymin=img_parsed.features.feature['image/object/bbox/ymin'].float_list.value[:]
ymax=img_parsed.features.feature['image/object/bbox/ymax'].float_list.value[:]
by=img_parsed.features.feature['image/by'].bytes_list.value[0].decode()
classes=img_parsed.features.feature['image/object/class/text'].bytes_list.value[:]
class_label=img_parsed.features.feature['image/object/class/label'].int64_list.value[:]
img_encoded=img_parsed.features.feature['image/encoded'].bytes_list.value[0]

In [36]:
# fig = plt.figure(figsize=(10,10))
# axes = axes = fig.add_subplot(1, 1, 1)
# img = Image.open(BytesIO(img_encoded))
# plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label, by)

Notice: putting jpeg-encoded images in TFRecords does not release the full potential of TFRecords. If you are training on a machine with fast I/O and lots of disk space, it is better to do all the preprocessing before storing the image as normalized float array. Let's define another function for this:

In [37]:
def create_tf_example2(imagedf, longest_edge=1024):  
    # Filename of the image (full path is useful when there are multiple image directories)
    fname = ARTAXOR_PATH+imagedf.file.iloc[0]  #exclude image path 
    filename=fname.split('/')[-1]              #exclude image name
    by = get_attribution(fname)                #exclude EXIF data
    img = Image.open(fname, "r")
    source_id = filename.split('.')[0]

    # resize image if larger that longest edge while keeping aspect ratio
    if max(img.size) > longest_edge:
        img.thumbnail((longest_edge, longest_edge), Image.ANTIALIAS)

    image_data = np.asarray(img)

    # storing shape will make it easy to reconstruct image later
    image_shape = np.array(image_data.shape)
    # convert to float
    image_data = image_data.reshape(image_data.shape[0] * image_data.shape[1] * image_data.shape[2])
    image_data = image_data.astype(float)/255. # normalize to [0,1]

    # object bounding boxes 
    # List of normalized coordinates in bounding box (1 per box)
    xmins, xmaxs, ymins, ymaxs = imagedf[['left', 'right', 'top', 'bottom']].values.T

    # List of string class name & id of bounding box (1 per box)
    classes_text = []
    classes = []
    for i in range(len(imagedf)):
        classes_text.append(imagedf.label.iloc[i].encode())
        classes.append(1+imagedf.label_idx.iloc[i])

    tf_record = tf.train.Example(features=tf.train.Features(feature={
        'image/shape': tf.train.Feature(int64_list=tf.train.Int64List(value=image_shape)),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[source_id.encode()])),
        'image/by': tf.train.Feature(bytes_list=tf.train.BytesList(value=[by.encode()])),
        'image/data': tf.train.Feature(float_list=tf.train.FloatList(value=image_data)),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes))
    }))
    return tf_record

In [38]:
sample_file2='ArTaxOr/Hymenoptera/ab30b4c2f70c.jpg'
imagedf=objectdf[objectdf.file == sample_file2]
tfr2 = create_tf_example2(imagedf)
fname2 = 'image_ex2.tfrecord'
with tf.io.TFRecordWriter(fname2) as writer:
    writer.write(tfr2.SerializeToString())
print(f"Size of {fname2} is {os.path.getsize(fname2)//1024} kbytes")

Size of image_ex2.tfrecord is 9768 kbytes


Notice: 

Storing image data as float gives a filesize of almost 10MB while the jpeg-encoded one was only 104kB. But now all preprocessing steps are complete, and the image can be fed directly to training (or possibly agumentation).

In [39]:
dataset2 = tf.data.TFRecordDataset(fname2)
img_example2 = next(iter(dataset2)) 
img_parsed2 = tf.train.Example.FromString(img_example2.numpy())
# extract features
xmin=img_parsed2.features.feature['image/object/bbox/xmin'].float_list.value[:]
xmax=img_parsed2.features.feature['image/object/bbox/xmax'].float_list.value[:]
ymin=img_parsed2.features.feature['image/object/bbox/ymin'].float_list.value[:]
ymax=img_parsed2.features.feature['image/object/bbox/ymax'].float_list.value[:]
by=img_parsed2.features.feature['image/by'].bytes_list.value[0].decode()
classes=img_parsed2.features.feature['image/object/class/text'].bytes_list.value[:]
class_label=img_parsed2.features.feature['image/object/class/label'].int64_list.value[:]
img_shape=img_parsed2.features.feature['image/shape'].int64_list.value[:]
img_data=img_parsed2.features.feature['image/data'].float_list.value[:]

In [40]:
image2=np.array(img_data).reshape(img_shape) # reshape
image2=image2*255. # scale back to [0, 255] and convert to int
image2=image2.astype(int)
img=Image.fromarray(np.uint8(image2))   ###????
# fig = plt.figure(figsize=(10,10))
# axes = axes = fig.add_subplot(1, 1, 1)
# plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label, by)

Now, let's consider the case when we have a huge dataset, and need to create multiple TFRecords.

In [41]:
filelist=pd.read_pickle(pickles+'ArTaxOr_filelist.pkl')
filelist=filelist.sample(frac=1)
filelist.head()

,format,id,name,path,size,state,type
6199,jpg,fe581fd8fd1a9f2126a5f805adf4a728,c43c4edff6db.jpg,ArTaxOr/Diptera/c43c4edff6db.jpg,"{'width': 790, 'height': 925}",2,1
3267,jpg,6afabf834ab4e4678b9a48661dcbd8fb,8a60682eaf27.jpg,ArTaxOr/Odonata/8a60682eaf27.jpg,"{'width': 2048, 'height': 1366}",2,1
7032,jpg,d3f7c5eb4885f6fcbe86005ba8bade79,22f73c720688.jpg,ArTaxOr/Lepidoptera/22f73c720688.jpg,"{'width': 5184, 'height': 3456}",2,1
5187,jpg,c2bcc2e6e1659db355f34f14ea77550f,3e31b19f381f.jpg,ArTaxOr/Diptera/3e31b19f381f.jpg,"{'width': 2048, 'height': 1536}",2,1
8299,jpg,d05a5b6709a571e7d902e02c8924846e,bde2bfd00ac8.jpg,ArTaxOr/Lepidoptera/bde2bfd00ac8.jpg,"{'width': 2048, 'height': 1589}",2,1


### The contextlib2 library
 is used to automatically close all TFRecords files after writing is finished.



In [56]:
%%time
import contextlib2

!mkdir /content/tensorflow-tfrecords-demystified/

########???????????????????????????
def open_sharded_tfrecords(exit_stack, base_path, num_shards):
    tf_record_output_filenames = [f'/content/tensorflow-tfrecords-demystified/{base_path}-{idx:05d}-of-{num_shards:05d}.tfrecord' for idx in range(num_shards)]

    tfrecords = [exit_stack.enter_context(tf.io.TFRecordWriter(file_name)) for file_name in tf_record_output_filenames]

    return tfrecords

num_shards=50
output_filebase='./ArTaxOr'

with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = open_sharded_tfrecords(tf_record_close_stack, output_filebase, num_shards)
    for i in range(len(filelist)):
        ldf=objectdf[objectdf.id == filelist.id.iloc[i]].reset_index()
        tf_record = create_tf_example(ldf, longest_edge=1280)
        output_shard_index = i % num_shards
        output_tfrecords[output_shard_index].write(tf_record.SerializeToString())

CPU times: user 37min 54s, sys: 19.3 s, total: 38min 13s
Wall time: 38min 54s


In [59]:
!ls -lh /content/tensorflow-tfrecords-demystified/ArTaxOr*.tfrecord

-rw-r--r-- 1 root root 39M Dec  9 07:56 /content/tensorflow-tfrecords-demystified/ArTaxOr-00000-of-00050.tfrecord
-rw-r--r-- 1 root root 42M Dec  9 07:56 /content/tensorflow-tfrecords-demystified/ArTaxOr-00001-of-00050.tfrecord
-rw-r--r-- 1 root root 40M Dec  9 07:56 /content/tensorflow-tfrecords-demystified/ArTaxOr-00002-of-00050.tfrecord
-rw-r--r-- 1 root root 41M Dec  9 07:56 /content/tensorflow-tfrecords-demystified/ArTaxOr-00003-of-00050.tfrecord
-rw-r--r-- 1 root root 39M Dec  9 07:56 /content/tensorflow-tfrecords-demystified/ArTaxOr-00004-of-00050.tfrecord
-rw-r--r-- 1 root root 40M Dec  9 07:56 /content/tensorflow-tfrecords-demystified/ArTaxOr-00005-of-00050.tfrecord
-rw-r--r-- 1 root root 40M Dec  9 07:56 /content/tensorflow-tfrecords-demystified/ArTaxOr-00006-of-00050.tfrecord
-rw-r--r-- 1 root root 42M Dec  9 07:56 /content/tensorflow-tfrecords-demystified/ArTaxOr-00007-of-00050.tfrecord
-rw-r--r-- 1 root root 41M Dec  9 07:56 /content/tensorflow-tfrecords-demystified/ArTaxO

To use TF Object Detection API, a label definition file is also needed


In [60]:
labels=pd.read_pickle(pickles+'ArTaxOr_labels.pkl')
pbfile=open('/content/tensorflow-tfrecords-demystified/ArTaxOr.pbtxt', 'w') 
for i in range(len(labels)): 
    pbfile.write(f"item {{\n id: {i+1}\n name:\'{labels.name[i]}\'\n}}\n\n")
pbfile.close()

In [61]:
## Read a few records from one of the .tfrecord files
# fname='./ArTaxOr-00029-of-00050.tfrecord' 
# dataset3 = tf.data.TFRecordDataset(fname)
# fig = plt.figure(figsize=(16,18))
# idx=1
# for raw_record in dataset3.take(6):
#     axes = fig.add_subplot(3, 2, idx)
#     example = tf.train.Example()
#     example.ParseFromString(raw_record.numpy())
#     xmin=example.features.feature['image/object/bbox/xmin'].float_list.value[:]
#     xmax=example.features.feature['image/object/bbox/xmax'].float_list.value[:]
#     ymin=example.features.feature['image/object/bbox/ymin'].float_list.value[:]
#     ymax=example.features.feature['image/object/bbox/ymax'].float_list.value[:]
#     by=example.features.feature['image/by'].bytes_list.value[0].decode()
#     classes=example.features.feature['image/object/class/text'].bytes_list.value[:]
#     class_label=example.features.feature['image/object/class/label'].int64_list.value[:]
#     img_encoded=example.features.feature['image/encoded'].bytes_list.value[0]
#     img = Image.open(BytesIO(img_encoded))
#     plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label, by)
#     idx=idx+1

# TF2 Object Detection API

## Install Object Detection API

In [62]:
import os
import pathlib
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [63]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696518 sha256=bfd65f13578cab29efc09098cb8dd6f5a2c17e2892354e7938dfe038d2941f9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-h14elixx/wheels/7d/96/c1/072a751379735e8dfdada1def1c62a89afb3cc45654fd6fd28
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [64]:
# # make sure we have the right tf version
# !pip install tensorflow==2.4.1
# !pip install tensorflow-addons

import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

### Hack to enable auto-augmentation
The auto-augmentation policy does not work in the current release, so a hack is made below to make it work.

In [65]:
# %cd /content/models/research/object_detection/utils
# !cp autoaugment_utils.py autoaugment_utils.py.org

# with open('autoaugment_utils.py.org', 'r') as f:
#     txt = f.read()

# # hack to make auto_augmentation work
# txt = txt.replace('from tensorflow.contrib import image as contrib_image', 'from tensorflow_addons import image as contrib_image')
# txt = txt.replace('from tensorflow.contrib import training as contrib_training', 'from collections import namedtuple')
# txt = txt.replace('# TODO(barretzoph): Add in ArXiv link once paper is out.', 'def hparams(**kwargs):\n    return namedtuple(\"HParams\", kwargs.keys())(*kwargs.values())\n\n')
# txt = txt.replace('augmentation_hparams = contrib_training.HParams(', 'augmentation_hparams = hparams(')

# with open('autoaugment_utils.py', 'w') as f:
#     f.write(txt)
# # copy the file to the installation directory
# !cp autoaugment_utils.py /lib/python3.7/site-packages/object_detection/utils/.
# %cd /content



## Install a model from the model garden
Choose from a large selection of object detection models in the [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md).

In [66]:
MODEL = "faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8"

%cd /content/models/research/object_detection
!mkdir pre-trained-models
%cd pre-trained-models
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/{MODEL}.tar.gz
!tar -xf {MODEL}.tar.gz
!rm {MODEL}.tar.gz

/content/models/research/object_detection
mkdir: cannot create directory ‘pre-trained-models’: File exists
/content/models/research/object_detection/pre-trained-models
--2022-12-09 08:02:29--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.194.128, 2404:6800:4003:c04::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.194.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212150153 (202M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8.tar.gz’

faster_rcnn_resnet5 100%[===================>] 202.32M  45.9MB/s    in 4.4s    

2022-12-09 08:02:34 (45.9 MB/s) - ‘faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8.tar.gz’ saved [212150153/212150153]



## Edit pipeline.config
The pipeline.config file is in protocol buffer format, and we can use the config_util to change the settings we need.

In [67]:
from object_detection.utils import config_util


from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [68]:
pipeline_config = f"./{MODEL}/pipeline.config"
model_dir = f"./{MODEL}/checkpoint"

config = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = config['model']

# detection_model = model_builder.build(model_config=model_config, is_training=False)

# ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
# ckpt.restore(os.path.join(model_dir, 'ckpt-0')).expect_partial()


model_config = config['model']
model_config.faster_rcnn.num_classes = 7
train_config = config['train_config']
train_config.fine_tune_checkpoint = "/content/models/research/object_detection/pre-trained-models/"+MODEL+"/checkpoint/ckpt-0"
# GPU memory limits batch size
train_config.batch_size = 2
train_config.use_bfloat16 = False
# some models use "fine_tune" rather than "detection" for fine-tuning
train_config.fine_tune_checkpoint_type = "detection"
# set auto-augmentation

del train_config.data_augmentation_options[:]

a1=train_config.data_augmentation_options.add()
a1.autoaugment_image.policy_name = "v2"

# training dataset (80%)
train_input_config = config['train_input_config']
train_input_config.label_map_path = "/content/tensorflow-tfrecords-demystified/ArTaxOr.pbtxt"
train_input_config.tf_record_input_reader.input_path[:] = ["/content/tensorflow-tfrecords-demystified/ArTaxOr-????0-of-00050.tfrecord",
                                                           "/content/tensorflow-tfrecords-demystified/ArTaxOr-????2-of-00050.tfrecord",
                                                           "/content/tensorflow-tfrecords-demystified/ArTaxOr-????3-of-00050.tfrecord",
                                                           "/content/tensorflow-tfrecords-demystified/ArTaxOr-????4-of-00050.tfrecord",
                                                           "/content/tensorflow-tfrecords-demystified/ArTaxOr-????5-of-00050.tfrecord",
                                                           "/content/tensorflow-tfrecords-demystified/ArTaxOr-????6-of-00050.tfrecord",
                                                           "/content/tensorflow-tfrecords-demystified/ArTaxOr-????7-of-00050.tfrecord",
                                                           "/content/tensorflow-tfrecords-demystified/ArTaxOr-????9-of-00050.tfrecord"]
# evaluation dataset (20%)                                                           
eval_input_config = config['eval_input_config']
eval_input_config.label_map_path = "/content/tensorflow-tfrecords-demystified/ArTaxOr.pbtxt"
eval_input_config.tf_record_input_reader.input_path[:] = ["/content/tensorflow-tfrecords-demystified/ArTaxOr-????1-of-00050.tfrecord",
                                                          "/content/tensorflow-tfrecords-demystified/ArTaxOr-????8-of-00050.tfrecord"]
# save pipeline.config file
pipeline_proto = config_util.create_pipeline_proto_from_configs(config)
config_util.save_pipeline_config(pipeline_config=pipeline_proto, 
                                 directory=f'/content/models/research/object_detection/pre-trained-models/{MODEL}')

mkdir: cannot create directory ‘/content/tensorflow-tfrecords-demystified/’: File exists


In [69]:
# Check the contents of the pipeline.config file:

!cat /content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config

model {
  faster_rcnn {
    num_classes: 7
    image_resizer {
      fixed_shape_resizer {
        height: 1024
        width: 1024
      }
    }
    feature_extractor {
      type: "faster_rcnn_resnet50_keras"
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        height_stride: 16
        width_stride: 16
        scales: 0.25
        scales: 0.5
        scales: 1.0
        scales: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 1.0
        aspect_ratios: 2.0
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.

## Training

In [ ]:
%cd /content/models/research
!python object_detection/model_main_tf2.py \
    --num_train_steps=1000 \
    --pipeline_config_path=/content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config \
    --model_dir=/content/ \
    --alsologtostderr &> /content/train.log

/content/models/research


In [ ]:
!tail -n 10 /content/train.log

    name_to_id = label_map_util.get_label_map_dict(
  File "/usr/local/lib/python3.8/dist-packages/object_detection/utils/label_map_util.py", line 201, in get_label_map_dict
    label_map = load_labelmap(label_map_path_or_proto)
  File "/usr/local/lib/python3.8/dist-packages/object_detection/utils/label_map_util.py", line 168, in load_labelmap
    label_map_string = fid.read()
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/lib/io/file_io.py", line 114, in read
    self._preread_check()
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/lib/io/file_io.py", line 76, in _preread_check
    self._read_buf = _pywrap_file_io.BufferedInputStream(
tensorflow.python.framework.errors_impl.NotFoundError: /content/tensorflow-tfrecords-demystified/ArTaxOr.pbtxt; No such file or directory


In [ ]:
# Plot the training curve by parsing the log file:


with open('/content/train.log') as f:
  data = f.read()

loss = []
for l in re.findall('loss=[.0-9]+', data):
  loss.append(float(l.split('=')[-1]))

step = []
for s in re.findall("Step [0-9]+", data):
    step.append(int(s.split(' ')[-1]))

plt.figure(figsize=(16, 8))
plt.plot(step,loss)
plt.legend(['model loss'])
plt.xlabel('Global step')
plt.title('Learning curve')

In [ ]:
#Evaluation step
!python object_detection/model_main_tf2.py \
    --pipeline_config_path=/content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config \
    --model_dir=/content/training/ \
    --checkpoint_dir=/content/training/ \
    --eval_timeout=30 \
    --alsologtostderr

In [ ]:
#Copy the piplie.config for later use
!cp /content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config /content/training/.

In [ ]:
#Export trained model
!python object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path=/kaggle/working/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config \
    --trained_checkpoint_dir /kaggle/working/training/ \
    --output_directory /kaggle/working/exported-models/effdet_d2

In [ ]:
!ls -al /kaggle/working/exported-models/effdet_d2/saved_model

In [ ]:
# clean-up 
%cd /kaggle/working
!rm -fr ./models